In [1]:
import os

import random
import pandas as pd

from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# import matplotlib.pyplot as plt
# import numpy as np

from PIL import Image


class DogClassifier:
    def __init__(self):
        self.model = self.build_model()
        # Add the class_names attribute
        self.class_names = ["Airedale", "Beagle", "Bloodhound", "Bluetick", "Chihuahua", "Collie", "Dingo",
                            "French Bulldog", "German Sheperd", "Malinois", "Newfoundland", "Pekinese",
                            "Pomeranian", "Pug", "Vizsla"]

    def build_model(self):
        '''
        Create a convolutional neural network model(CNN) :
        With one separate input layer, three convolutional layers and a maximum pooling layer
        '''

        model = Sequential()

        model.add(Input(shape=(224, 224, 3)))

        model.add(Conv2D(32, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))

        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))

        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))

        # Flattenning 3D feature maps into 1D vectors
        model.add(Flatten())

        # Add full connectivity layer
        model.add(Dense(512, activation='relu'))

        # Add Dropout layer to prevent overfitting
        model.add(Dropout(0.5))

        # Output layer, outputs classification results(there are 15 dog breed categories)
        model.add(Dense(15, activation='softmax'))

        # compiling the model
        model.compile(optimizer='adam', loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Printed Model Structures
        # model.summary()
        return model

    def train(self, train_dir, valid_dir, batch_size, epochs):

        # Preprocessing and enhancement of training and validation data
        train_datagen = ImageDataGenerator(rescale=1./255)
        valid_datagen = ImageDataGenerator(rescale=1./255)

        # Generating data streams for training and validation sets
        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(224, 224),
            batch_size=batch_size,
            class_mode='categorical'
        )

        valid_generator = valid_datagen.flow_from_directory(
            valid_dir,
            target_size=(224, 224),
            batch_size=batch_size,
            class_mode='categorical'
        )

        # Training the model
        self.model.fit(
            train_generator,
            steps_per_epoch=train_generator.samples // batch_size,
            epochs=epochs,
            validation_data=valid_generator,
            validation_steps=valid_generator.samples // batch_size,
        )

    def evaluate(self, valid_dir, batch_size):

        # Preprocessing and enhancement of validation data
        valid_datagen = ImageDataGenerator(rescale=1./255)

        # Generating data streams for validation sets
        valid_generator = valid_datagen.flow_from_directory(
            valid_dir,
            target_size=(224, 224),
            batch_size=batch_size,
            class_mode='categorical'
        )

        # Calculating accuracy
        scores = self.model.evaluate(
            valid_generator, steps=valid_generator.samples // batch_size)
        validation_accuracy = scores[1] * 100

        print("\nValid set Accuracy: %.2f%%\n" % validation_accuracy)

        # Output validation accuracy into Excel(no need title)
        df = pd.DataFrame({'Accuracy': [validation_accuracy]})
        df.to_excel('validation_accuracy.xlsx', index=False)

        print("Validation accuracy saved to validation_accuracy.xlsx")

    def center_crop_image(self, img):
        '''
        Pre-processing of images, with different sized images as input,
          and centered cropped images as output.
        '''

        width, height = img.size
        new_width = new_height = min(width, height)
        left = (width - new_width) // 2
        top = (height - new_height) // 2
        right = (width + new_width) // 2
        bottom = (height + new_height) // 2

        return img.crop((left, top, right, bottom))

    def get_predicted_breed(self, predictions):
        '''
        To get the dog breed name based on the model predictions,
        assume that predictions is the result of the model's prediction of the image,
        determine the predicted breed based on the index of the highest probability in the probability vector
        '''

        breed_index = predictions.argmax()
        breed_name = self.class_names[breed_index]

        return breed_name

    def test(self, test_dir):

        # Getting test set file address
        test_files = os.listdir(test_dir)
        # To be fair, the order of access is randomized.
        random.shuffle(test_files)

        test_results = []

        for file_name in test_files:

            # Load image with center crop and preprocessing
            img_path = os.path.join(test_dir, file_name)
            img = Image.open(img_path)
            # Center Cropped Image
            img = self.center_crop_image(img)
            # resize
            img = img.resize((224, 224))
            img_array = img_to_array(img)
            img_array = preprocess_input(img_array)
            img_array = img_array.reshape((1,) + img_array.shape)

            # Making predictions about the image
            predictions = self.model.predict(img_array)
            predicted_breed = self.get_predicted_breed(predictions)

            test_results.append((file_name, predicted_breed))

        # Output results into Excel(no need title)
        df = pd.DataFrame(test_results, columns=[
                          'File Name', 'Predicted Breed'])
        df.to_excel('test_data.xlsx', index=False, header=False)
        print("Test results saved to test_data.xlsx")


def main():
    classifier = DogClassifier()
    train_dir = 'archive/train'
    valid_dir = 'archive/valid'
    test_dir = 'archive/testing_set'
    batch_size = 32
    epochs = 10

    classifier.train(train_dir, valid_dir, batch_size, epochs)
    classifier.evaluate(valid_dir, batch_size)
    classifier.test(test_dir)


if __name__ == "__main__":
    main()

Found 1714 images belonging to 15 classes.
Found 150 images belonging to 15 classes.
Epoch 1/10


d:\python\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


53/53 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - accuracy: 0.0931 - loss: 3.4966 - val_accuracy: 0.1875 - val_loss: 2.3928
Epoch 2/10
 1/53 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.1875 - loss: 2.4629

d:\python\lib\contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1875 - loss: 1.2547 - val_accuracy: 0.1818 - val_loss: 1.1873
Epoch 3/10
Epoch 3/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.2002 - loss: 2.3549 - val_accuracy: 0.3047 - val_loss: 2.1949
Epoch 4/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2812 - loss: 1.1506 - val_accuracy: 0.3182 - val_loss: 0.9703
Epoch 5/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.3126 - loss: 2.0649 - val_accuracy: 0.2969 - val_loss: 2.1441
Epoch 6/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4375 - loss: 1.0641 - val_accuracy: 0.3636 - val_loss: 1.0548
Epoch 7/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.4560 - loss: 1.7225 - val_accuracy: 0.3438 - val_loss: 2.0157
Epoch 8/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4688 - loss: 0.8492 - val_accuracy: 0.5909 - val_loss: 0.7243
Epoch 9/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.6512 - loss: 1.2055 - val_accuracy: 0.4375 - val_loss: